# YamboConvergence: automated BSE convergence

The highest level workflow is represented by the ``YamboConvergence`` workchain, 
which implements the full automation of the convergence algorithm described in [Bonacci, M., Qiao, J., Spallanzani, N. et al. Towards high-throughput many-body perturbation theory: efficient algorithms and automated workflows. npj Comput Mater 9, 74 (2023)](https://doi.org/10.1038/s41524-023-01027-2). 
Simulations are organized on the fly, without any external user intervention. 
The purpose of this new proposed convergence algorithm is to obtain an accurate converged 
result doing the least possible number of calculations. This is possible if a reliable description of the convergence space is achieved, resulting also in a 
precise guess for the converged point, i.e. the converged parameters. The description of the space is performed by fitting some calculations that the workchain runs. 
A simple functional form of the space is assumed:

$f(\textbf{x}) = \prod_i^N \left( \frac{A_i}{x_i^{\alpha_i}} + b_i \right)$

where N is the number of parameters which we are converging simultaneously.
In this way it is straightforward to compute first and second partial derivatives, and impose constraints on them to find the converged region of the space. 
The algorithm can be used to easily perform convergence tests with respect to the ``k-point mesh``.

Each simulation is performed by calling the `YamboWorkflow` workchain, ensuring the automation at all levels.

In [1]:
from aiida import orm, load_profile
load_profile()

Profile<uuid='b35700dae723411ea16ebc82d58f16bc' name='mb'>

In [2]:
from aiida.plugins import WorkflowFactory

In [3]:
YamboConvergence = WorkflowFactory('yambo.yambo.yamboconvergence')

In [ ]:
from aiida.orm import QueryBuilder
qb = QueryBuilder()
qb.append(orm.Group, filters={'label': 'hBN/bulk'}, tag='group')
qb.append(orm.StructureData, with_group='group')

loaded_structure_id = qb.all()[0][0].pk

In [ ]:
import yaml,io

# Read YAML file
with open("../configuration/codes_localhost.yaml", 'r') as stream:
    codes = yaml.safe_load(stream)
    
with open("../configuration/resources_localhost.yaml", 'r') as stream:
    resources = yaml.safe_load(stream)

## minimal inputs needed for protocols

In [4]:
options = {
    'pseudo_family':"PseudoDojo/0.4/PBE/SR/standard/upf",
    'protocol':'fast',
    #'parent_id':274, #not necessary to set; take your previously nscf id (pk) to skip the DFT part.
    'structure_id':loaded_structure_id,
}

In [ ]:
options.update(codes)

In [5]:
from aiida_quantumespresso.common.types import ElectronicType

In [6]:
builder = YamboConvergence.get_builder_from_protocol(
            pw_code = options['pwcode_id'],
            preprocessing_code = options['yamboprecode_id'],
            code = options['yambocode_id'],
            protocol=options['protocol'],
            protocol_qe=options['protocol'],
            structure=orm.load_node(options['structure_id']),
            overrides={},
            #parent_folder=load_node(options['parent_id']).outputs.remote_folder,
            electronic_type=ElectronicType.INSULATOR, #default is METAL: smearing is used
            calc_type='bse', #or 'bse'; default is 'gw'
)

Summary of the main inputs:
BndsRnXs = 200
NGsBlkXs = 6 Ry
BSENGBlk = 6 Ry
FFTGvecs = 18 Ry


kpoint mesh for nscf: [6, 6, 2]


/scratch/mbonacci/mambaforge/envs/minitutorial/lib/python3.10/site-packages/aiida_quantumespresso/calculations/pw.py:211: UserWarning: `parent_folder` not provided for `nscf` calculation. For work chains wrapping this calculation, you can disable this warning by excluding the `parent_folder` when exposing the inputs of the `PwCalculation`.
  warnings.warn(


In [7]:
#You can also try different protocols:
    
YamboConvergence.get_available_protocols()

{'fast': {'description': 'Fast protocol for a GW convergence: grid -> poor; thresholds -> poor'},
 'moderate': {'description': 'Moderate protocol for a GW convergence: grid -> enough good for standard materials; thresholds -> moderate (5 percent)'},
 'precise': {'description': 'precise protocol for a GW convergence: grid -> same as moderate; thresholds -> precise (1 percent)'},
 'molecule': {'description': 'Moderate protocol for a GW convergence in molecules'}}

Now, if you inspect the prepopulated inputs, you can see the default values respecting the imposed protocol:

In [8]:
builder.ywfl.nscf.pw.parameters.get_dict()

{'CONTROL': {'calculation': 'nscf',
  'forc_conv_thr': 0.001,
  'tprnfor': True,
  'tstress': True,
  'etot_conv_thr': 0.0004},
 'SYSTEM': {'nosym': False,
  'occupations': 'fixed',
  'ecutwfc': 60.0,
  'ecutrho': 480.0,
  'force_symmorphic': True,
  'nbnd': 200},
 'ELECTRONS': {'electron_maxstep': 80,
  'mixing_beta': 0.4,
  'conv_thr': 1.6e-09}}

In [30]:
builder.ywfl.yres.yambo.parameters.get_dict()

{'arguments': ['em1s', 'bse', 'bss', 'optics', 'rim_cut', 'dipoles'],
 'variables': {'Chimod': 'hartree',
  'DysSolver': 'n',
  'GTermKind': 'BG',
  'X_and_IO_nCPU_LinAlg_INV': [1, ''],
  'BSEmod': 'resonant',
  'BSKmod': 'SEX',
  'BSSmod': 'd',
  'Lkind': 'full',
  'BSEQptR': [[1, 1], ''],
  'FFTGvecs': [18, 'Ry'],
  'GbndRnge': [[1, 200], ''],
  'BndsRnXs': [[1, 200], ''],
  'NGsBlkXs': [6, 'Ry'],
  'BSENGBlk': [6, 'Ry']}}

### Computational resources


In [31]:
builder.ywfl.scf.pw.metadata.options = resources

builder.ywfl.nscf.pw.metadata.options = builder.ywfl.scf.pw.metadata.options
builder.ywfl.yres.yambo.metadata.options = builder.ywfl.scf.pw.metadata.options

### Overrides

It is possible to modify the default inputs also during the builder creation phase, so not a posteriori. This can be done by using overrides:

In [32]:
overrides_scf = {
        'pseudo_family': "PseudoDojo/0.4/PBE/SR/standard/upf", 
        'pw':{
            
        'metadata':resources,
        },
    }

overrides_nscf = {
        'pseudo_family': "PseudoDojo/0.4/PBE/SR/standard/upf", 
        'pw': {
            'parameters':{
                'CONTROL':{}, #not needed if you don't override something
                'SYSTEM':{},
                'ELECTRONS':{'diagonalization':'cg'},
            },
             'metadata':resources,
    },
}

overrides_yambo = {
        "yambo": {
            "parameters": {
                "arguments": [
                    "rim_cut",
                ],
                "variables": {
                    "FFTGvecs": [20, "Ry"],
                    'GbndRnge': [[1, 50], ''],
                    'BndsRnXs': [[1, 50], ''],
                    'NGsBlkXs': [2, 'Ry'],
                    'BSENGBlk': [2, 'Ry'],
                    'KfnQP_E':[[1.5,1,1],''],           # <== Scissor and stretching correction.
                    'BS_CPU':str(int(16/2))+' 2 1',     # <== PARALLELISM INFO
                    'BS_ROLEs':'k eh t',                # <== PARALLELISM INFO
                },
            },
        'metadata':resources,
        },
    
}

#Be careful with the mesh choice!!! 
overrides_meta = {
        'FFTGvecs': {
            'start_ratio': 0.25,
            'stop_ratio': 0.7,
            'delta_ratio': 0.1,
            'max_ratio': 1,
        },
        'bands': {
            'start': 50,
            'stop': 400,
            'delta': 50,
            'max': 600,
            'ratio':[10,25,50],
        },
        'G_vectors': {
            'start': 2,
            'stop': 8,
            'delta': 1,
            'max': 10,
        },
        'kpoint_density': {
            'start': 0.8,
            'stop': 0.2,
            'delta': 3,
            'max': 0.1,
        } ,
        'conv_thr_k': 5,   # <== 5% for convergence wrt super-converged estimation.
        'conv_thr_bG': 10,
        'conv_thr_FFT': 10,
        'conv_thr_units': '%', # 'eV'

        
    }

        
overrides_wfl_settings = {
        
        'what':['lowest_exciton'],          # <== converging the lowest exciton.
        'bands_nscf_update':'full-step',
        'skip_pre':False,   
        'type': 'heavy', #or cheap; heavy uses converged value for parameters that we are not converging in a given iteration.
        
    }

overrides = {
        'meta_parameters':overrides_meta,
        'ywfl':{'scf':overrides_scf,'nscf':overrides_nscf,'yres':overrides_yambo},
        'workflow_settings':overrides_wfl_settings,
    }


In [33]:
builder = YamboConvergence.get_builder_from_protocol(
            pw_code = options['pwcode_id'],
            preprocessing_code = options['yamboprecode_id'],
            code = options['yambocode_id'],
            protocol=options['protocol'],
            protocol_qe=options['protocol'],
            structure= orm.load_node(options['structure_id']),
            overrides=overrides,
            #parent_folder=load_node(options['parent_id']).outputs.remote_folder,
            electronic_type=ElectronicType.INSULATOR, #default is METAL: smearing is used
            calc_type='bse', #or 'bse'; default is 'gw'
)

family = orm.load_group("PseudoDojo/0.4/PBE/SR/standard/upf")
#builder.<sublevels_up_to .pw>.pseudos = family.get_pseudos(structure=structure) 
builder.ywfl.scf.pw.pseudos = family.get_pseudos(structure=orm.load_node(loaded_structure_id)) 
builder.ywfl.nscf.pw.pseudos = family.get_pseudos(structure=orm.load_node(loaded_structure_id)) 

Summary of the main inputs:
BndsRnXs = 50
NGsBlkXs = 2 Ry
BSENGBlk = 2 Ry
FFTGvecs = 20 Ry


kpoint mesh for nscf: [6, 6, 2]


/scratch/mbonacci/mambaforge/envs/minitutorial/lib/python3.10/site-packages/aiida_quantumespresso/calculations/pw.py:211: UserWarning: `parent_folder` not provided for `nscf` calculation. For work chains wrapping this calculation, you can disable this warning by excluding the `parent_folder` when exposing the inputs of the `PwCalculation`.
  warnings.warn(


In [34]:
try:
    g = orm.load_group('tutorial/hBN/convergence/BSE')
except:
    g = orm.Group('tutorial/hBN/convergence/BSE')
    g.store()

In [35]:
builder.group_label = orm.Str('tutorial/hBN/convergence/BSE') # verdi group create tutorial/hBN/convergence; all calculationsc are added to the group

In [36]:
builder.ywfl.yres.yambo.parameters = orm.Dict(
    dict={'arguments':['em1s','bse','bss','optics', 'dipoles',],
                'variables':{
                'BSEmod': 'resonant',
                'BSKmod': 'SEX',
                'BSSmod': 'd',
                'Lkind': 'full',
                'NGsBlkXs': [2, 'Ry'],
                'BSENGBlk': [2, 'Ry'],
                'Chimod': 'hartree',
                'DysSolver': 'n',
                'BEnSteps': [10,''],
                'BSEQptR': [[1,1],''],
                'BSEBands': [[8,9],''],
                'BEnRange': [[0.0, 10.0],'eV'],
                'BDmRange': [[0.1, 0.1],'eV'],
                'BLongDir': [[1.0, 1.0, 1.0],''],
                'LongDrXp': [[1.0, 1.0, 1.0],''],
                'LongDrXd': [[1.0, 1.0, 1.0],''],
                'LongDrXs': [[1.0, 1.0, 1.0],''],
                'BndsRnXs': [[1,50], ''],
                'KfnQP_E':[[1.5,1,1],''],
                'BS_CPU':str(int(16/2))+' 2 1',
                'BS_ROLEs':'k eh t',
                },}
)

## Solution: setting the kpoint density as convergence parameter

Be careful on the parameters: we provide 1/x values, where x is the actual distance between kpoints in reciprocal space (in 1/A). 

In [42]:
builder.parameters_space = orm.List(list=[
 {'var': ['kpoint_density'],
  'start': 1/0.8,
  'stop': 1/0.4,
  'delta': 0.5,      #1/0.01
  'max': 1/0.2,
  'steps': 4,
  'max_iterations': 4,
  'conv_thr': 3,
  'conv_thr_units': '%',
  'convergence_algorithm': 'new_algorithm_1D'},])

In [43]:
builder.parameters_space.get_list()


[{'var': ['kpoint_density'],
  'start': 1.25,
  'stop': 2.5,
  'delta': 0.5,
  'max': 5.0,
  'steps': 4,
  'max_iterations': 4,
  'conv_thr': 3,
  'conv_thr_units': '%',
  'convergence_algorithm': 'new_algorithm_1D'}]

### Overrides

It is possible to modify the default inputs also during the builder creation phase, so not a posteriori. This can be done by using overrides:

In [44]:
from aiida.engine import submit

In [45]:
run = None

In [46]:
if run:
    print('run is already running -> {}'.format(run.pk))
    print('sure that you want to run again?, if so, copy the else instruction in the cell below and run!')
else:
    run = submit(builder)

print(run)

/scratch/mbonacci/mambaforge/envs/minitutorial/lib/python3.10/site-packages/aiida_quantumespresso/calculations/pw.py:211: UserWarning: `parent_folder` not provided for `nscf` calculation. For work chains wrapping this calculation, you can disable this warning by excluding the `parent_folder` when exposing the inputs of the `PwCalculation`.
  warnings.warn(


uuid: 8cbe303e-d774-4737-ad0e-799bd308a208 (pk: 7233) (aiida.workflows:yambo.yambo.yamboconvergence)


# Output analysis.

suppose that your calculation completed successfully, then you can access the outputs via the output method of the run instance: 

In [14]:
run.is_finished_ok

True

The converged parameters can be obtained via the "infos" output Dict:

In [15]:
run.outputs.infos.get_dict()

{'E_ref': 5.575053237994,
 'kpoint_density': 1.75,
 'lowest_exciton': 5.5288791656494}

The full convergence history can be visualized in a table form using pandas:

In [16]:
import pandas as pd

In [17]:
history = run.outputs.history.get_dict()

In [18]:
history_table = pd.DataFrame(history)

In [19]:
history_table

,uuid,failed,useful,global_step,kpoint_density,lowest_exciton,parameters_studied
0,1b01b60f-bf26-4c4a-be35-d145547e635c,False,False,1,1.25,5.359427,kpoint_density
1,cf3a389a-a35b-4cfb-8190-a8712920eb55,False,True,2,1.75,5.528879,kpoint_density
2,4c0ff3fd-863a-43a8-aa51-f0d4f5bf887d,False,False,3,2.25,5.533526,kpoint_density
3,24dd9bbb-69bf-4b04-a867-592d32917abc,False,False,4,2.50,5.533520,kpoint_density


The last calculations can be obtained using:

In [20]:
history_table[history_table['useful']==True]

,uuid,failed,useful,global_step,kpoint_density,lowest_exciton,parameters_studied
1,cf3a389a-a35b-4cfb-8190-a8712920eb55,False,True,2,1.75,5.528879,kpoint_density


Result on the convergence path can be plotted using several plotting libraries, for examples here we are gonna use [plotly]((https://plotly.com/python/3d-line-plots/)) to observe the convergence between bands and plane wave cutoff for the screening matrix:

In [23]:
import plotly.express as px

k_history = history_table[history_table['parameters_studied']=="kpoint_density"]

df = k_history
fig = px.scatter(df,x ="kpoint_density" , y="lowest_exciton")
fig.show()